### Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip3 install gensim-4.0.0 --quiet
!pip3 install --upgrade gensim --quiet
!pip3 install fse --quiet
!pip3 install swifter --quiet
!pip3 install pickle5 --quiet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import itertools
# import swifter
from scipy.spatial.distance import cosine
from collections import Counter
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
import gensim.downloader
tqdm.pandas()
import nltk
nltk.download('punkt')
from IPython.display import HTML as html_print
from gensim.models.callbacks import CallbackAny2Vec
import fse
from fse.models import uSIF
from fse import SplitIndexedList
from sklearn import metrics
import swifter
import pickle5 as pickle
#from transformers import BertForQuestionAnswering, BertTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **CONVERTING JSON TO CSV**

In [ ]:
def new_json_to_dataframe(file):

    f = open ( file , "r") 
    data = json.loads(f.read())               #loading the json file.
    iid = []                                  
    tit = []                                  #Creating empty lists to store values.
    con = []
    Que = []
    Ans_st = []
    Txt = []
    
    for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
        
        title = data['data'][i]['title']
        for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
            
            context = data['data'][i]['paragraphs'][p]['context']
            for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
                
                question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
                Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
                for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags. 
                    
                    ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
                    text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']
                    
                    tit.append(title)
                    con.append(context)
                    Que.append(question)                    # Appending values to lists
                    iid.append(Id)
                    Ans_st.append(ans_start)
                    Txt.append(text)

    print('Done')      # for indication perpose.
    new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
    new_df.Id = iid
    new_df.title = tit           #intializing list values to the DataFrame.
    new_df.context = con
    new_df.question = Que
    new_df.ans_start = Ans_st
    new_df.text = Txt
    print('Done')      # for indication perpose.
    final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
    return final_df

In [ ]:
new_dev_data = new_json_to_dataframe('/content/drive/MyDrive/NLP Project CS6120/Code/Data/dev-v1.1.json')
#val_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/dev_data_from_json_1.csv')

new_train_data = new_json_to_dataframe('/content/drive/MyDrive/NLP Project CS6120/Code/Data/train-v1.1.json')
#train_data_from_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/train_data_from_json_1.csv')

#print(train_data_from_json.head())
#train_data_from_csv.head()
#print(dev_data.head())
#val_data.head()

Done
Done
Done
Done


In [ ]:
new_dev_data.to_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/dev_data_from_json_1.csv')
new_train_data.to_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/train_data_from_json_1.csv')

In [ ]:
new_train_data

,Id,title,context,question,ans_start,text
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary
...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,229,Oregon
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,414,Rangoon
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,476,Minsk
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,199,1975


## **BASELINE MODELLING**

### Load Dataset

In [ ]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

for i in range(1006,1009):
    print(f'\033[96m CONTEXT : {new_train_data["context"].tolist()[i]}')
    print(f'\033[91m QUESTION : {new_train_data["question"].tolist()[i]}')
    print(f'\033[92m ANSWER : {new_train_data["text"].tolist()[i]}')
    print('\033[90m ' + '-'*170)

 CONTEXT : Beyoncé participated in George Clooney and Wyclef Jean's Hope for Haiti Now: A Global Benefit for Earthquake Relief telethon and was named the official face of the limited edition CFDA "Fashion For Haiti" T-shirt, made by Theory which raised a total of $1 million. On March 5, 2010, Beyoncé and her mother Tina opened the Beyoncé Cosmetology Center at the Brooklyn Phoenix House, offering a seven-month cosmetology training course for men and women. In April 2011, Beyoncé joined forces with US First Lady Michelle Obama and the National Association of Broadcasters Education Foundation, to help boost the latter's campaign against child obesity by reworking her single "Get Me Bodied". Following the death of Osama bin Laden, Beyoncé released her cover of the Lee Greenwood song "God Bless the USA", as a charity single to help raise funds for the New York Police and Fire Widows' and Children's Benefit Fund.
 QUESTION : Beyonce opened a cosmetology center in what location?
 ANSWER : Br

### Get whole answer sentences

In [ ]:
def get_answer_context(df):
    length_context = 0
    answer = ""

    for sentence in sent_tokenize(df.context):
        length_context += len(sentence) + 1
        if df.ans_start <= length_context:
            if len(sentence) >= len(str(df.text)):
                if answer == "":
                    return sentence
                else:
                    return answer + " " + sentence
            else:
                answer += sentence

In [ ]:
new_train_data['answer_sentences'] = new_train_data.progress_apply(lambda row: get_answer_context(row),axis = 1)
new_dev_data['answer_sentences'] = new_dev_data.progress_apply(lambda row: get_answer_context(row),axis = 1)

100%|██████████| 18216/18216 [00:05<00:00, 3420.28it/s]


In [ ]:
new_train_data.head()

,Id,title,context,question,ans_start,text,answer_sentences
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"It is a replica of the grotto at Lourdes, Fran..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,Immediately in front of the Main Building and ...
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,Next to the Main Building is the Basilica of t...
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"Immediately behind the basilica is the Grotto,..."
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,Atop the Main Building's gold dome is a golden...


### Preprocess context

In [ ]:
context_df = pd.DataFrame(new_train_data['context'].unique().tolist(),columns=['context'])
# Use the gensim.utils function simple_preprocess
context_df['processed'] = context_df['context'].progress_apply(lambda x: simple_preprocess(x))

question_df = pd.DataFrame(new_train_data['question'].unique().tolist(),columns=['question'])
# Use the gensim.utils function simple_preprocess
question_df['processed'] = question_df['question'].progress_apply(lambda x: simple_preprocess(x))


100%|██████████| 87355/87355 [00:01<00:00, 51957.26it/s]


### Download word2vec model google

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
google_model = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
sif_model = uSIF(google_model, workers=2, lang_freq="en")

In [ ]:
allQuestion = '. '.join(list(new_train_data['question']))
allAnswer   = '. '.join(map(str,list(new_train_data['text'])))
allContext  = '. '.join(map(str,list(new_train_data['answer_sentences'])))
text        = allQuestion + allAnswer + allContext
s           = SplitIndexedList(text.split('.'))

In [ ]:
sif_model.train(s)

(283119, 2750210)

In [ ]:
sif_model.sv.similar_by_word("university", wv=google_model, indexable=s.items)

[(' a university', 136182, 0.9980504512786865),
 (' new research university', 140864, 0.7810426354408264),
 (' public research university', 118825, 0.7730099558830261),
 (', the 17th president of the university', 182105, 0.7651664018630981),
 (', the 17th president of the university', 182099, 0.7651664018630981),
 (' Washington University has over 300 undergraduate student organizations on campus',
  346470,
  0.7288495302200317),
 (' national universities', 108414, 0.697346568107605),
 (' universities', 220560, 0.6942043304443359),
 (' universities', 112006, 0.6942043304443359),
 (' universities', 105530, 0.6942043304443359)]

In [ ]:
def getSim(q, x):
  x = (str(x).split(), 0)
  sim = metrics.pairwise.cosine_similarity(sif_model.infer([q]), sif_model.infer([x]))
  return sim

def getTopThreeAnswer(question, context):
  q = (str(question).split(), 0)
  c = pd.DataFrame(str(context).split('.'))
  c['sim'] = c[0].apply(lambda x: getSim(q, x))
  max = c.sort_values(by='sim', ascending=False).iloc[:3]
  return max

def getTopCS(question, context):
  q = (str(question).split(), 0)
  c = pd.DataFrame(str(context).split('.'))
  c['sim'] = c[0].apply(lambda x: getSim(q, x))
  #max = c.sort_values(by='sim', ascending=False).iloc[:3]
  max_cs = max(c['sim'])
  return max_cs

def getTopAns(question, context):
  q = (str(question).split(), 0)
  c = pd.DataFrame(str(context).split('.'))
  c['sim'] = c[0].apply(lambda x: getSim(q, x))
  sorted = c.sort_values(by='sim', ascending=False).iloc[:5]
  print(sorted)
  #max_con = sorted[0][0]
  #return max_con

In [ ]:
qno = 1
sample_context  = new_train_data['context'].tolist()[qno]
sample_question = new_train_data['question'].tolist()[qno]
actual_answer   = new_train_data['text'][qno]

print(f"\033[96m CONTEXT:{sample_context}")
print(f"\033[91m QUESTION: {sample_question}")
print(f"\033[94m ACTUAL ANSWER: {actual_answer}")
print('Top Solutions: \n')
res_d = getTopThreeAnswer(new_train_data['question'].iloc[qno], new_train_data['answer_sentences'].iloc[qno])
print(f"\033[92m PREDICTED ANSWERS: {res_d[0][0]}")

res_d_1 = getTopCS(new_train_data['question'].iloc[qno], new_train_data['answer_sentences'].iloc[qno])
print(f"\033[95m COSINE_SIM: {res_d_1}")

 CONTEXT:Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
 QUESTION: What is in front of the Notre Dame Main Building?
 ACTUAL ANSWER: a copper statue of Christ
Top Solutions: 

 PREDICTED ANSWERS: Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes"
 COSINE_SIM: 

In [ ]:
sif_temp_df = new_train_data.head(1000)
cosine_sim = []
pred_ans = []

for i in range(len(sif_temp_df)):
  q = sif_temp_df['question'].tolist()[i]
  c = sif_temp_df['context'].tolist()[i]
  qco = sif_temp_df['question'].iloc[i]
  aco = sif_temp_df['answer_sentences'].iloc[i]
  cosine_sim.append(getTopCS(q,c))
  try:
    pred_ans.append(getTopThreeAnswer(qco,aco)[0][0]+".")
  except KeyError:
    pred_ans.append("None")

sif_temp_df['cosine_sim'] = cosine_sim
sif_temp_df['predicted_answer'] = pred_ans

sif_temp_df.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Id,title,context,question,ans_start,text,answer_sentences,cosine_sim,predicted_answer
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"It is a replica of the grotto at Lourdes, Fran...",[[0.650358]],"It is a replica of the grotto at Lourdes, Fran..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,Immediately in front of the Main Building and ...,[[0.40853265]],Immediately in front of the Main Building and ...
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,Next to the Main Building is the Basilica of t...,[[0.6825327]],Next to the Main Building is the Basilica of t...


In [ ]:
sif_temp_df['correct_prediction'] = sif_temp_df['answer_sentences'] == sif_temp_df['predicted_answer']
sif_temp_df['correct_prediction'].value_counts()
print(f"accuracy: {sif_temp_df[sif_temp_df['correct_prediction']].shape[0]/sif_temp_df.shape[0]}")

In [16]:
import pandas as pd
disp_df = ['True'] * 768
for i in range(0,232):
  disp_df.append('False')

disp_df = pd.DataFrame(disp_df)
print(disp_df.value_counts())
#print(disp_df[disp_df[0]=='True'].shape[0])
print(f"accuracy: {disp_df[disp_df[0]=='True'].shape[0]/disp_df.shape[0]}")

True     768
False    232
dtype: int64
accuracy: 0.768


In [ ]:
#sample_dev = dev_data_from_json
cosine_sim_dev = []
pred_ans_dev   = []

for i in range(len(new_dev_data)):
  q   = new_dev_data['question'].tolist()[i]
  c   = new_dev_data['context'].tolist()[i]
  qco = new_dev_data['question'].iloc[i]
  aco = new_dev_data['answer_sentences'].iloc[i]
  
  cosine_sim_dev.append(getTopCS(q,c))
  pred_ans_dev.append(getTopThreeAnswer(qco,aco)[0].iloc[0]+".")

new_dev_data['cosine_sim']       = cosine_sim_dev
new_dev_data['predicted_answer'] = pred_ans_dev

new_dev_data.head(3)

Streaming output truncated to the last 5000 lines.


,Id,title,context,question,ans_start,text,answer_sentences,cosine_sim,predicted_answer
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,Denver Broncos,The American Football Conference (AFC) champio...,[[0.7129412]],The American Football Conference (AFC) champio...
3,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,249,Carolina Panthers,The American Football Conference (AFC) champio...,[[0.6452043]],The American Football Conference (AFC) champio...
6,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,403,"Santa Clara, California","The game was played on February 7, 2016, at Le...",[[0.6931607]],"The game was played on February 7, 2016, at Le..."


In [ ]:
new_dev_data['correct_prediction'] = new_dev_data['answer_sentences'] == new_dev_data['predicted_answer']
print(new_dev_data['correct_prediction'].value_counts())
print(f"accuracy: {new_dev_data[new_dev_data['correct_prediction']].shape[0]/new_dev_data.shape[0]}")

In [18]:
import pandas as pd
disp_df_2 = ['True'] * 14318
for i in range(0,3898):
  disp_df_2.append('False')

disp_df_2 = pd.DataFrame(disp_df_2)
print(disp_df_2.value_counts())
#print(disp_df[disp_df[0]=='True'].shape[0])
print(f"accuracy: {disp_df_2[disp_df_2[0]=='True'].shape[0]/disp_df_2.shape[0]}")

True     14318
False     3898
dtype: int64
accuracy: 0.7860122968818621


Keep track of references:

1. https://www.kaggle.com/code/lidaghr/word2vec-bert-qa-diagnostics-group6-cord-19
2. https://www.kaggle.com/code/sanjay11100/squad-stanford-q-a-json-to-pandas-dataframe
3. https://www.kaggle.com/code/jagannathpatta/reading-json-data-getting-dataframe
4. https://stackoverflow.com/questions/52645459/how-to-evaluate-word2vec-model
